In [1]:
%pip install PyMuPDF
%pip install tqdm
%pip install spacy
%pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 114.7 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
matplotlib 3.8.2 requires numpy<2,>=1.21, but you have numpy 2.2.6 which is incompatible.
pandas 2.1.4 requires numpy<2,>=1.22.4; python_version < "3.11", but you have numpy 2.2.6 which is incompatible.
scikit-learn 1.3.2 requires numpy<2.0,>=1.17.3, but you have numpy 2.2.6 which is incompatible.
scipy 1.11.4 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.2.6 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
   ━━━━

In [1]:
from huggingface_hub import login
login()

In [1]:
!nvidia-smi

Thu Sep 18 07:00:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:05.0 Off |                    0 |
| N/A   43C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Importing the Document


In [3]:
import os
import requests

pdf_path = "human-nutrition-text.pdf"

if not os.path.exists(pdf_path):
  print("[INFO] File doesn't exist, downloading ...")
  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"
  filename = pdf_path
  res = requests.get(url)
  if res.status_code == 200:
    with open(filename, "wb") as file:
      file.write(res.content)
    print(f"[INFO] File has been downloaded and saved as {filename}")
  else:
    print(f"[INFO] Failed to download the file {res.status_code}")
else:
  print(f"File {pdf_path} exists")

File human-nutrition-text.pdf exists


Opening the PDF

In [4]:
import pymupdf
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
  cleaned_text = text.replace("\n"," ").strip()
  return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
  doc = pymupdf.open(pdf_path)
  pages_and_texts = []
  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()
    text = text_formatter(text=text)
    pages_and_texts.append({"page_number":page_number-41,
                            "page_char_count": len(text),
                            "page_word_count": len(text.split(" ")),
                            "page_sentence_count_raw": len(text.split(". ")),
                            "page_token_count":len(text)/4,
                            "text":text})
  return pages_and_texts
pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [5]:
import random

random.sample(pages_and_texts, k=3)

[{'page_number': 443,
  'page_char_count': 1822,
  'page_word_count': 308,
  'page_sentence_count_raw': 17,
  'page_token_count': 455.5,
  'text': 'Effects of Alcohol Abuse on the Brain  A small amount (up to 10%) of the liver acetaldehyde may  accumulate inside the liver cells. As more alcohol is ingested, this  stimulates the production of acetaldehyde by both the alcohol  dehydrogenase and MEOS systems. As the levels of acetaldehyde  increase inside the liver cells with heavy consumption of alcohol,  some of the acetaldehyde diffuse into the blood circulation. In  circulation, high levels of acetaldehyde cause nausea and vomiting.  Vomiting causes more body dehydration and loss of electrolytes.  If the dehydration becomes severe enough, this can impair brain  function and a person may lose consciousness.  Alcohol can adversely affect nearly every area of the brain. When  BAC rises, the central nervous system is depressed. Alcohol disrupts  the way nerve cells communicate with each o

In [6]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [7]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.004139,198.299669,9.972682,287.001035
std,348.86387,560.382275,95.759336,6.187226,140.095569
min,-41.00000,0.000000,1.000000,1.000000,0.000000
25%,260.75000,762.000000,134.000000,4.000000,190.500000
50%,562.50000,1231.500000,214.500000,10.000000,307.875000
75%,864.25000,1603.500000,271.000000,14.000000,400.875000
max,1166.00000,2308.000000,429.000000,32.000000,577.000000


Splitting pages into sentences

In [8]:
from spacy.lang.en import English

nlp = English()
nlp.add_pipe("sentencizer")

for item in tqdm(pages_and_texts):
  item["sentences"] = list(nlp(item["text"]).sents)
  item["sentences"] = [str(sentence) for sentence in item["sentences"]]
  item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [9]:
random.sample(pages_and_texts, k=1)

[{'page_number': 735,
  'page_char_count': 1303,
  'page_word_count': 225,
  'page_sentence_count_raw': 10,
  'page_token_count': 325.75,
  'text': 'Allergy Warnings  Food manufacturers are required by the FDA to list on their  packages if the product contains any of the eight most common  ingredients that cause food allergies. These eight common allergens  are as follows: milk, eggs, peanuts, tree nuts, fish, shellfish, soy, and  wheat. (More information on these allergens will be discussed in  Chapter 18 “Nutritional Issues”.) The FDA does not require warnings  that cross contamination may occur during packaging, however  most manufacturers include this advisory as a courtesy. For  instance, you may notice a label that states, “This product is  manufactured in a factory that also processes peanuts.” If you have  food allergies, it is best to avoid products that may have been  contaminated with the allergen.  When Enough Is Enough  Estimating Portion Size  Have you ever heard the expr

In [10]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,198.30,9.97,287.00,10.32
std,348.86,560.38,95.76,6.19,140.10,6.30
min,-41.00,0.00,1.00,1.00,0.00,0.00
25%,260.75,762.00,134.00,4.00,190.50,5.00
50%,562.50,1231.50,214.50,10.00,307.88,10.00
75%,864.25,1603.50,271.00,14.00,400.88,15.00
max,1166.00,2308.00,429.00,32.00,577.00,28.00


Chunking our sentences together

In [11]:
num_sentence_chunk_size = 10

def split_list(input_list: list[str], slice_size: int=num_sentence_chunk_size) -> list[list[str]]:
  return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]

test_list = list(range(25))
split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [12]:
for item in tqdm(pages_and_texts):
  item["sentence_chunks"] = split_list(item["sentences"])
  item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [13]:
random.sample(pages_and_texts, k=1)

[{'page_number': 789,
  'page_char_count': 1661,
  'page_word_count': 309,
  'page_sentence_count_raw': 15,
  'page_token_count': 415.25,
  'text': 'Table 13.1 Body Mass Index and Pregnancy3  Prepregnancy BMI Weight Category  Recommended Weight Gain  Below 18.5  Underweight  28–40 lbs.  18.5–24.9  Normal  25–35 lbs.  25.0–29.9  Overweight  15–25 lbs.  Above 30.0  Obese (all classes) 11–20 lbs.  Starting weight below or above the normal range can lead to  different complications. Pregnant women with a prepregnancy BMI  below twenty are at a higher risk of a preterm delivery and an  underweight infant. Pregnant women with a prepregnancy BMI  above thirty have an increased risk for a cesarean section during  delivery. Therefore, it is optimal to have a BMI in the normal range  prior to pregnancy.  Generally, women gain 2 to 5 pounds in the first trimester. After  that, it is best not to gain more than one pound per week. Some of  the new weight is due to the growth of the fetus, while som

In [14]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,198.30,9.97,287.00,10.32,1.53
std,348.86,560.38,95.76,6.19,140.10,6.30,0.64
min,-41.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,260.75,762.00,134.00,4.00,190.50,5.00,1.00
50%,562.50,1231.50,214.50,10.00,307.88,10.00,1.00
75%,864.25,1603.50,271.00,14.00,400.88,15.00,2.00
max,1166.00,2308.00,429.00,32.00,577.00,28.00,3.00


Splitting each chunk into its own item

In [15]:
import re

pages_and_chunks = []
for item in tqdm(pages_and_texts):
  for sentence_chunk in item["sentence_chunks"]:
    chunk_dict= {}
    chunk_dict["page_number"] = item["page_number"]
    joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
    joined_sentence_chunk = re.sub(r'\.([A-Z])',r'. \1',joined_sentence_chunk)
    chunk_dict["sentence_chunk"] = joined_sentence_chunk
    chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
    chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
    chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4
    pages_and_chunks.append(chunk_dict)

  0%|          | 0/1208 [00:00<?, ?it/s]

In [16]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 362,
  'sentence_chunk': 'An interactive or media element has been excluded from this version of the text. You can view it online here: http://pressbooks.oer.hawaii.edu/ humannutrition2/?p=239 \xa0 362 | Introduction',
  'chunk_char_count': 187,
  'chunk_word_count': 26,
  'chunk_token_count': 46.75}]

In [17]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,583.38,734.44,112.33,183.61
std,347.79,447.54,71.22,111.89
min,-41.00,12.00,3.00,3.00
25%,280.50,315.00,44.00,78.75
50%,586.00,746.00,114.00,186.50
75%,890.00,1118.50,173.00,279.62
max,1166.00,1831.00,297.00,457.75


Filter chunks of text for short chunks

In [18]:
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
  print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 12.5 | Text: Figure 11.2 The Structure of Hemoglobin Iron | 655
Chunk token count: 26.0 | Text: http://www.ncbi.nlm.nih.gov/pubmed/20182023. Accessed September 22, 2017. 220 | Popular Beverage Choices
Chunk token count: 5.5 | Text: 1050 | Comparing Diets
Chunk token count: 9.75 | Text: 920 | Older Adulthood: The Golden Years
Chunk token count: 4.5 | Text: 708 | Introduction


In [19]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient = "records")
random.sample(pages_and_chunks_over_min_token_len, k=1)

[{'page_number': 16,
  'sentence_chunk': 'USDA https://ww w.ars.usda.gov/ northeast- area/ beltsville- md-bhnrc/ beltsville-h uman-nutr ition-resea rch-center / food-surve ys-researc h-group/ docs/ whats-in-t he-foods-y ou-eat-ems earch-toole m/ Learning Activities Technology Note: The second edition of the Human Nutrition Open Educational Resource (OER) textbook features interactive learning activities.\xa0 These activities are available in the web-based textbook and not available in the 16 | Food Quality',
  'chunk_char_count': 466,
  'chunk_word_count': 58,
  'chunk_token_count': 116.5}]

In [20]:
len(pages_and_chunks_over_min_token_len)

1680

## Embeddings


In [21]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(model_name_or_path='google/embeddinggemma-300M', device='cuda')

In [6]:
from huggingface_hub import snapshot_download
import os

# This will return the local cached path if already downloaded
local_dir = snapshot_download(
    repo_id="google/embeddinggemma-300M",
    local_files_only=True   # ensures no internet is used
)

print("Model is stored at:", local_dir)

Model is stored at: /teamspace/studios/this_studio/.cache/huggingface/hub/models--google--embeddinggemma-300M/snapshots/c5cfa06e5e282a820e85d57f7fb053207494f41d


In [25]:
# for item in tqdm(pages_and_chunks_over_min_token_len):
#   item["embedding"] = model.encode(item["sentence_chunk"]).tolist()

In [26]:
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

In [27]:
text_chunk_embedding = model.encode(text_chunks, show_progress_bar=True, batch_size=64)

Batches:   0%|          | 0/27 [00:00<?, ?it/s]

In [28]:
text_chunk_embedding.shape

(1680, 768)

In [29]:
for item, embedding in tqdm(zip(pages_and_chunks_over_min_token_len, text_chunk_embedding)):
  item["embedding"] = embedding

0it [00:00, ?it/s]

In [30]:
random.sample(pages_and_chunks_over_min_token_len, k=1)

[{'page_number': 949,
  'sentence_chunk': 'An interactive or media element has been excluded from this version of the text. You can view it online here: http://pressbooks.oer.hawaii.edu/ humannutrition2/?p=496 \xa0 An interactive or media element has been excluded from this version of the text. You can view it online here: http://pressbooks.oer.hawaii.edu/ humannutrition2/?p=496 The Benefits of Physical Activity | 949',
  'chunk_char_count': 375,
  'chunk_word_count': 52,
  'chunk_token_count': 93.75,
  'embedding': array([-2.00592652e-02,  7.74877751e-03, -2.27282885e-02, -1.23532712e-02,
         -7.70966634e-02, -1.11455964e-02,  2.15141959e-02,  2.71797273e-02,
          1.01011675e-02, -1.04587391e-01,  1.60172712e-02, -2.71236431e-02,
          8.78304914e-02, -1.39682218e-02,  6.80590421e-02,  5.77848172e-03,
         -1.21288300e-02,  1.00732431e-01,  1.86430160e-02, -5.83923869e-02,
         -3.53719667e-02,  4.37033921e-02,  1.85000105e-03,  7.09343255e-02,
          6.273822

Save Embeddings to file

In [31]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_path, index=False)

Importing the Embeddings

In [32]:
text_chunks_and_embeddings_df_load = pd.read_csv(embeddings_df_path)
text_chunks_and_embeddings_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,[-1.73992962e-02 -7.86828157e-03 -1.68744064e-...
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,[-5.85181117e-02 -2.83373706e-02 2.33737007e-...
2,-37,Contents Preface University of Hawai‘i at Māno...,766,114,191.50,[ 1.46887731e-02 -6.84199063e-03 1.44355884e-...
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,142,235.25,[ 3.54745351e-02 1.09032737e-02 3.61862965e-...
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,[-1.10343806e-02 8.57283128e-04 1.19341677e-...


## RAG - Search and Answer

Similarity Search


In [1]:
import random
import numpy as np
import torch
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
text_chunks_and_embeddings_df = pd.read_csv("text_chunks_and_embeddings_df.csv")

text_chunks_and_embeddings_df["embedding"] = text_chunks_and_embeddings_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

embeddings = torch.tensor(np.stack(text_chunks_and_embeddings_df["embedding"].to_list(), axis=0, dtype=np.float32)).to(device)

pages_and_chunks = text_chunks_and_embeddings_df.to_dict(orient="records")

In [3]:
text_chunks_and_embeddings_df["embedding"]

0       [-0.0173992962, -0.00786828157, -1.68744064e-0...
1       [-0.0585181117, -0.0283373706, 0.0233737007, 0...
2       [0.0146887731, -0.00684199063, 0.0144355884, 0...
3       [0.0354745351, 0.0109032737, 0.0361862965, 0.0...
4       [-0.0110343806, 0.000857283128, 0.0119341677, ...
                              ...                        
1675    [-0.0192632601, -0.012909105, -0.0615346171, 0...
1676    [-0.0263409652, 0.0148344897, -0.0873417333, 0...
1677    [-0.0384976, 0.02612713, -0.05804822, 0.031823...
1678    [-0.037653286, 0.0253833029, -0.095193617, 0.0...
1679    [-0.0491830148, -0.0193542372, -0.0502372421, ...
Name: embedding, Length: 1680, dtype: object

In [4]:
embeddings.shape

torch.Size([1680, 768])

In [5]:
from sentence_transformers import util, SentenceTransformer

model = SentenceTransformer("google/embeddinggemma-300M", device=device)

Sample pipeline

1. Define Query.
2. Embed Query.
3. Perform dot product or cosine similarity b/w text and query embedding.
4. Sort the results from 3 in descending order.

In [6]:
query = "Nutrition for child growth"
query_embedding = model.encode(query, convert_to_tensor=True, device=device)
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
top_results_dot_scores = torch.topk(dot_scores, k=5)
top_results_dot_scores

torch.return_types.topk(
values=tensor([0.5735, 0.5478, 0.5352, 0.5335, 0.5227], device='cuda:0'),
indices=tensor([1137, 1237, 1091, 1234, 1212], device='cuda:0'))

In [7]:
import textwrap

def print_wrapped(text, wrap_length=80):
    print(textwrap.fill(text, wrap_length))

In [8]:
print(f"Query: {query}\n")
print("Results:\n")
for score, idx in zip(top_results_dot_scores[0], top_results_dot_scores[1]):
    print(f"\nScore: {score:.4f}")
    print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
    print(f"Page number: {pages_and_chunks[idx]['page_number']}")

Query: Nutrition for child growth

Results:


Score: 0.5735
Image by Marie Kainoa Fialkowski Revilla / CC BY 4.0 Infancy UNIVERSITY OF
HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM AND HUMAN NUTRITION
PROGRAM Diet and nutrition have a major impact on a child’s development from
infancy into the adolescent years. A healthy diet not only affects growth, but
also immunity, intellectual capabilities, and emotional well-being. One of the
most important jobs of parenting Infancy | 807
Page number: 807

Score: 0.5478
Parents and caregivers should select a variety of foods from each food group to
ensure that nutritional requirements are met. Because children grow rapidly,
they require foods that are high in iron, such as lean meats, legumes, fish,
poultry, and iron-enriched cereals. Adequate fluoride is crucial to support
strong teeth. One of the most important micronutrient requirements during
childhood is adequate calcium and vitamin D intake. Both are needed to build
dense bones 

### Functionalizing the semantic search pipeline

In [9]:
def retrieve_relevant_resources(query: str, 
                                embeddings: torch.tensor, 
                                model: SentenceTransformer=model,
                                n_resources_to_return: int=5):
    
    query_embedding = model.encode(query, convert_to_tensor=True).to('cuda')
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    scores, indices = torch.topk(dot_scores, k=n_resources_to_return)
    return scores, indices

def print_top_result_and_scores(query: str,
                                embeddings: torch.tensor=embeddings,
                                text: list[dict]=pages_and_chunks,
                                n_resources_to_return: int=5):
    
    scores, indices = retrieve_relevant_resources(query, embeddings)
    print(f"Query: {query}\n")
    print("Results:\n")
    for score, idx in zip(scores, indices):
        print(f"\nScore: {score:.4f}")
        print_wrapped(text[idx]["sentence_chunk"])
        print(f"Page number: {text[idx]['page_number']}")
    return

In [10]:
query = "Foods high in fiber"
print_top_result_and_scores(query)

Query: Foods high in fiber

Results:


Score: 0.5134
• Change it up a bit and experience the taste and satisfaction of other whole
grains such as barley, quinoa, and bulgur. • Eat snacks high in fiber, such as
almonds, pistachios, raisins, and air-popped popcorn. Add an artichoke and green
peas to your dinner plate more 276 | Carbohydrates and Personal Diet Choices
Page number: 276

Score: 0.5103
Dietary fiber is categorized as either water-soluble or insoluble. Some examples
of soluble fibers are inulin, pectin, and guar gum and they are found in peas,
beans, oats, barley, and rye. Cellulose and lignin are insoluble fibers and a
few dietary sources of them are whole-grain foods, flax, cauliflower, and
avocados. Cellulose is the most abundant fiber in plants, making up the cell
walls and providing structure. Soluble fibers are more easily accessible to
bacterial enzymes in the large intestine so they can be broken down to a greater
extent than insoluble fibers, but even some breakdown 

## LLM

In [11]:
!pip install -U transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="google/gemma-3-4b-it",
    device="cuda",
    torch_dtype=torch.bfloat16
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


In [53]:
output = pipe(text_inputs="How often should infants be breastfed?", max_new_tokens=1000)

print(output[0]["generated_text"])

 How often should infants be breastfed?</h1>

The World Health Organization (WHO) recommends exclusive breastfeeding for the first six months of life. This means giving the baby only breast milk, with no other liquids or foods.

After six months, you can introduce complementary foods alongside breast milk. However, breastfeeding should continue for at least two years, and ideally longer, for optimal health and development.

Here’s a more detailed breakdown of breastfeeding frequency:

*   **First 6 Months:** Breastfeed on demand, which typically means whenever the baby shows signs of hunger. This can be every 1-3 hours, or even more frequently for newborns.
*   **6-12 Months:** Continue breastfeeding on demand, but you might start to notice a more predictable feeding schedule. Most babies will breastfeed 4-6 times in a 24-hour period.
*   **12 Months and Beyond:** Breastfeeding is still beneficial and can be continued as long as both the mother and baby desire. Frequency will vary depe

In [17]:
output = pipe(text_inputs="How often should infants be breastfed?",return_full_text=False, max_new_tokens=1000)

print(output[0]["generated_text"])



The World Health Organization (WHO) recommends that infants should be breastfed exclusively for the first six months of life. After six months, breastfeeding should continue alongside the introduction of complementary foods.

Here’s a more detailed breakdown of feeding frequency:

**0-6 Months (Exclusive Breastfeeding)**

*   **On Demand:** This is the key principle. Breastfeed whenever your baby shows signs of hunger.
*   **Frequency:** Newborns typically feed every 2-3 hours, sometimes more frequently. This can mean 8-12 feedings in a 24-hour period.
*   **Duration:** Feed for as long as your baby wants, allowing them to fully drain the breast. This can range from 10-45 minutes per feeding.
*   **Signs of Hunger:** Rooting (turning their head and opening their mouth as if searching for the breast), sucking on hands or fingers, lip smacking, fussiness, and restlessness.

**6 Months Onward (Breastfeeding + Complementary Foods)**

*   **Frequency:** The frequency will decrease as your

In [12]:
model = pipe.model

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

Total parameters: 4,300,079,472
Trainable parameters: 4,300,079,472


In [15]:
input_text = "How often should infants be breastfed?"
message_template = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": input_text}
        ]
    }
]

In [16]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it")
prompts = tokenizer.apply_chat_template(message_template, tokenize=False, add_generation_prompt=True)
print(prompts)
input_ids = tokenizer(prompts, return_tensors="pt").to("cuda")
print(input_ids)
output = model.generate(**input_ids, max_new_tokens=1000)
print(output[0])
output_text = tokenizer.decode(output[0])
print(output_text)

<bos><start_of_turn>user
How often should infants be breastfed?<end_of_turn>
<start_of_turn>model

{'input_ids': tensor([[     2,      2,    105,   2364,    107,   3910,   3187,   1374,  45033,
            577,  16489,  42168, 236881,    106,    107,    105,   4368,    107]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}
tensor([     2,      2,    105,   2364,    107,   3910,   3187,   1374,  45033,
           577,  16489,  42168, 236881,    106,    107,    105,   4368,    107,
         19058, 236764,   1531, 236789, 236751,   2541,   1679,   1217,   3187,
         45033,   1374,    577,  16489,  42168, 236761,   2085, 236789, 236751,
           951,   3161,    623,  47734,   1548,   2098,    618,    625,  22130,
         14562,  10167,    580,    506,   7205, 236789, 236751,   3911, 236764,
         39485, 236764,    532,   3141,   3548, 236761,   3153, 236764,   1590,
        236789, 236751,    496

In [20]:
# Nutrition-style questions generated with GPT4
gpt4_questions = [
    "What are the macronutrients, and what roles do they play in the human body?",
    "How do vitamins and minerals differ in their roles and importance for health?",
    "Describe the process of digestion and absorption of nutrients in the human body.",
    "What role does fibre play in digestion? Name five fibre containing foods.",
    "Explain the concept of energy balance and its importance in weight management."
]

# Manually created question list
manual_questions = [
    "How often should infants be breastfed?",
    "What are symptoms of pellagra?",
    "How does saliva help with digestion?",
    "What is the RDI for protein per day?",
    "water soluble vitamins"
]

query_list = gpt4_questions + manual_questions

In [21]:
import random

query = random.choice(query_list)
print(f"Question: {query}\n")
scores, indices = retrieve_relevant_resources(query, embeddings)
scores, indices

Question: What are the macronutrients, and what roles do they play in the human body?



(tensor([0.5759, 0.5509, 0.5367, 0.5352, 0.5229], device='cuda:0'),
 tensor([ 42,   7,  46,  41, 381], device='cuda:0'))

### Augmentation

In [22]:
def prompt_formatter(query: str,
                    context_items: list[dict])->list[dict]:
    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])
    base_prompt = """Based on the following context items, please answer the query.
    Give yourself room to think by extracting relevant passages from the context before answering the query.
    Don't return the thinking, only return the answer.
    Make sure your answers are as explanatory as possible.
    Use the following context items to answer the user query:
    {context}
    \nRelevant passages: <extract relevant passages from the context here>
    User query: {query}
    Answer:"""
    prompt = base_prompt.format(context=context,query=query)
    return prompt

In [23]:
query = random.choice(query_list)

scores, indices = retrieve_relevant_resources(query, embeddings)
context_items = [pages_and_chunks[i] for i in indices]
prompt = prompt_formatter(query, context_items)

In [56]:
output = pipe(prompt,
             max_new_tokens=200,
             return_full_text=False)

In [57]:
print(output[0]["generated_text"])

 The water-soluble vitamins are vitamin C and all the B vitamins, which include thiamine, riboflavin, niacin, pantothenic acid, pyridoxine, biotin, folate and cobalamin.




In [58]:
print(query)

water soluble vitamins


In [24]:
def ask(query: str,
        max_new_tokens: int=1000):
    scores, indices = retrieve_relevant_resources(query, embeddings)
    print(f"[INFO] {max_new_tokens}")
    context_items = [pages_and_chunks[i] for i in indices]
    prompt = prompt_formatter(query, context_items)
    answer = pipe(prompt,
             max_new_tokens=max_new_tokens,
             return_full_text=False)
    return answer[0]["generated_text"], context_items, prompt

In [25]:
query = "How often should infants be breastfed?"
print(f"Query: {query}\n")

answer, context_items, prompt = ask(query=query) 
print(f"Answer:\n")
print_wrapped(answer)
print(f"Context items:")
context_items

Query: How often should infants be breastfed?

[INFO] 1000
Answer:

 Newborns breastfeed 8-12 times/day. Between 1 and 3 months of age, the
breastfed infant becomes more efficient, and the number of feedings per day
often become fewer even though the amount of milk consumed stays the same.
Context items:


[{'page_number': 841,
  'sentence_chunk': 'Age Human Milk or infant formula Grain products Vegetables Fruit Protein-rich foods Birth – 6 months Newborns breastfeed 8-12 times/day. Formula fed infants should consume 2-3 ounces of formula every 3-4 hours and by 6 months consume 32 ounces/day. None None None None 6-8 months Breastfed infants continue to breastfeed, on demand. Formula-fed infants take in about 24-32 ounces. Amounts vary based on individual assessment. Intake of human milk or formula may decrease as complementary foods increase. About 1-2 ounces iron-fortified infant cereals, bread, small pieces of crackers About 2-4 ounces of cooked, plain, strained/ pureed/ mash vegetables About 2-4 ounces of plain strained/ pureed/ mashed fruits About 1-2 ounces meat, poultry, fish, eggs, cheese, yogurt, or legumes; all are plain strained/ pureed/ mashed Infancy | 841',
  'chunk_char_count': 835,
  'chunk_word_count': 127,
  'chunk_token_count': 208.75,
  'embedding': array([ 6.20658211e

In [30]:
prompt = """Use the following context items to answer the user query:
    - Age Human Milk or infant formula Grain products Vegetables Fruit Protein-rich foods Birth – 6 months Newborns breastfeed 8-12 times/day. Formula fed infants should consume 2-3 ounces of formula every 3-4 hours and by 6 months consume 32 ounces/day. None None None None 6-8 months Breastfed infants continue to breastfeed, on demand. Formula-fed infants take in about 24-32 ounces. Amounts vary based on individual assessment. Intake of human milk or formula may decrease as complementary foods increase. About 1-2 ounces iron-fortified infant cereals, bread, small pieces of crackers About 2-4 ounces of cooked, plain, strained/ pureed/ mash vegetables About 2-4 ounces of plain strained/ pureed/ mashed fruits About 1-2 ounces meat, poultry, fish, eggs, cheese, yogurt, or legumes; all are plain strained/ pureed/ mashed Infancy | 841
- milk is the best source to fulfill nutritional requirements. An exclusively breastfed infant does not even need extra water, including in hot climates. A newborn infant (birth to 28 days) requires feedings eight to twelve times a day or more. Between 1 and 3 months of age, the breastfed infant becomes more efficient, and the number of feedings per day often become fewer even though the amount of milk consumed stays the same. After about six months, infants can gradually begin to consume solid foods to help meet nutrient needs. Foods that are added in addition to breastmilk are called complementary foods. Complementary foods should be nutrient dense to provide optimal nutrition. Complementary foods include baby meats, vegetables, fruits, infant cereal, and dairy products such as yogurt, but not infant formula. Infant formula is a substitute, not a complement to breastmilk. In addition to complementary foods, the World Health Organization recommends that breastfeeding continue up to 2 years of age or beyond, and the American Academy of Pediatrics recommends at least one year of breastfeeding, or longer.
- Most babies need about 2.5 ounces of formula per pound of body weight each day. Therefore, the average infant consumes about 24 fluid ounces of breastmilk or formula per day, or one ounce per hour. If an infant sleeps for 2 hours, they often consume 2 ounces Infancy | 835
- American Academy of Pediatrics. American Academy of Pediatrics. (2012). Breastfeeding and the use of human milk. Pediatrics, 129(3), e827. American Academy of Pediatrics. (2018. September 24). Amount and schedule of formula feedings.https://www.healthychildren.org/ English/ages-stages/baby/formula-feeding/Pages/Amount-and- Schedule-of-Formula-Feedings.aspx Leonberg, B. L. (2020).
- compares the recommended vitamins and minerals for lactating women to the levels for nonpregnant and pregnant women. Table 13.3 Recommended Nutrient Intakes during Lactation 824 | Infancy
    
    User query: How often should infants be breastfed?"""

In [31]:
output = pipe(text_inputs=prompt,return_full_text=False, max_new_tokens=1000)

print(output[0]["generated_text"])


    Context items:
    - Age Human Milk or infant formula Grain products Vegetables Fruit Protein-rich foods Birth – 6 months Newborns breastfeed 8-12 times/day. Formula fed infants should consume 2-3 ounces of formula every 3-4 hours and by 6 months consume 32 ounces/day. None None None None 6-8 months Breastfed infants continue to breastfeed, on demand. Formula-fed infants take in about 24-32 ounces. Amounts vary based on individual assessment. Intake of human milk or formula may decrease as complementary foods increase. About 1-2 ounces iron-fortified infant cereals, bread, small pieces of crackers About 2-4 ounces of cooked, plain, strained/ pureed/ mash vegetables About 2-4 ounces of plain strained/ pureed/ mashed fruits About 1-2 ounces meat, poultry, fish, eggs, cheese, yogurt, or legumes; all are plain strained/ pureed/ mashed Infancy | 841
    - milk is the best source to fulfill nutritional requirements. An exclusively breastfed infant does not even need extra water, includi